pandas: 데이터를 쉽고 유연하게 처리하기 위한 라이브러리입니다.

rdkit: 화학 정보학과 기계 학습을 위한 오픈 소스 화학 정보학 소프트웨어 툴킷입니다.
1. rdkit.Chem: 분자와 관련된 기본적인 기능과 클래스를 제공합니다.
2. rdkit.Chem.MACCSkys: 화합물의 분자 지문을 생성하는 데 사용되는 방법 중 하나입니다.

In [ ]:
import pandas as pd

try: 
    from rdkit import Chem
    from rdkit.Chem import MACCSkeys
    
except:
    import sys
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "rdkit-pypi"])
    # subprocess.check_call([sys.executable, "-m", "conda", "install", "rdkit", "-c conda-forge"])
    
    from rdkit import Chem
    from rdkit.Chem import MACCSkeys


smiles2fing 함수는 SMILES 형식의 문자열 리스트를 입력받아, 각 분자에 대한 MACCS 키 기반의 분자 fingerprint를 생성하고 처리합니다.


smiles2fing 함수의 동작 과정은 아래와 같습니다.
1. smiles2fing 함수의 입력으로 받아온 SMILES 문자열 리스트(smiles)는 Chem.MolFromSmiles 함수를 통해 각 문자열에 해당하는 분자 객체로 반환됩니다.
    - Chem.MolFromSmiles에 의해 반환되지 못하는 문자열은, None으로 반환됩니다.

2. None으로 반환된 모든 문자열들의 인덱스를 새로운 리스트 'ms_none_idx'에 저장합니다.
3. ms_tmp에서 None이 아닌(분자 객체로 성공적으로 반환된 친구들) 분자 객체만을 필터링하여 새로운 리스트 'ms'에 저장합니다.

4. ms에 속하는 모든 객체를 각각 MACCSkeys.GenMACCSKeys 함수를 통해 MACCS키로 변환하고, 이를 새로운 리스트 'maccs'에 저장합니다.
5. maccs의 모든 각 원소를 이진 문자열로 변환하고, 이를 새로운 리스트 'maccs_bit'에 저장합니다.

6. maccs_bit를 pandas 데이터프레임으로 변환합니다.
    - pandas.DataFrame을 통해 생성된 데이터프레임은 'maccs'라는 하나의 열을 가집니다.

7. str.split 메서드를 사용해 이진 문자열을 개별 문자 단위로 분할합니다.
    - expand 옵션이 True이기 때문에 분할된 개별 문자가 데이터프레임의 별도 열로 확장됩니다.
    - 단, 최대 167개의 문자로 분할될 수 있습니다.

8. 이진 문자열을 분할하면 첫 번째 열은 항상 비어있는 값으로 채워집니다. 해당 열은 불필요하기 때문에 데이터프레임에서 제거합니다.
9. 데이터프레임의 열의 이름을 maccs_1, maccs_2, maccs_3, ..., maccs_167로 지정합니다.
10. 데이터프레임의 데이터 타입을 정수로 변환하고, 인덱스를 재설정합니다.
11. 함수의 output으로 None으로 반환된 문자열들의 인덱스와 각종 처리가 수행된 fingerprint 데이터프레임을 반환합니다.

In [ ]:
def smiles2fing(smiles):
    ms_tmp = [Chem.MolFromSmiles(i) for i in smiles]
    ms_none_idx = [i for i in range(len(ms_tmp)) if ms_tmp[i] == None]
    
    ms = list(filter(None, ms_tmp))
    
    maccs = [MACCSkeys.GenMACCSKeys(i) for i in ms]
    maccs_bit = [i.ToBitString() for i in maccs]
    
    fingerprints = pd.DataFrame({'maccs': maccs_bit})
    fingerprints = fingerprints['maccs'].str.split(pat = '', n = 167, expand = True)
    fingerprints.drop(fingerprints.columns[0], axis = 1, inplace = True)
    
    colname = ['maccs_' + str(i) for i in range(1, 168)]
    fingerprints.columns = colname
    fingerprints = fingerprints.astype(int).reset_index(drop = True)
    
    return ms_none_idx, fingerprints